<a href="https://colab.research.google.com/github/profliuhao/CSIT355/blob/main/CSIT_355_Module_4_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4: SQL Queries

### [name]
### [student ID]

## Instruction
You need to finish all questions of both In-class exercise. Name your ipynb file as StudentName_lab2.ipynb and submit it on Canvas.

PLEASE ENSURE THAT THE SCRIPT CAN CORRECTLY EXECUTE!!!

In [1]:
# Let's start with load the extension sql
%load_ext sql

# and then connect to a database **lab4_SQL.db** using:
%sql sqlite:///lab4_SQL.db

In [2]:
%%sql
-- Create the "Sailors" table
drop table if exists Sailors;
drop table if exists Reserves;
drop table if exists Boats;
CREATE TABLE Sailors (
    sid int PRIMARY KEY,
    sname VARCHAR,
    rating int,
    age real
);

-- Create the "Reserves" table
CREATE TABLE Reserves (
    sid int,
    bid int,
    day DATE,
    PRIMARY KEY (sid, bid),
    FOREIGN KEY (sid) REFERENCES Sailors(sid),
    FOREIGN KEY (bid) REFERENCES Boats(bid)
);

-- Create the "Boats" table
CREATE TABLE Boats (
    bid int PRIMARY KEY,
    name VARCHAR,
    color VARCHAR
);

-- Insert rows into the "Sailors" table
delete from Sailors;
INSERT INTO Sailors (sid, sname, rating, age) VALUES
    (22, 'Dustin', 7, 45),
    (29, 'Brutus', 1, 33),
    (31, 'Lubber', 8, 55.5),
    (32, 'Andy', 8, 25.5),
    (58, 'Rusty', 10, 35),
    (64, 'Horatio', 7, 35),
    (71, 'Zorba', 10, 16),
    (74, 'Horatio', 9, 35),
    (85, 'Art', 3, 25.5),
    (95, 'Bob', 3, 63.5);

-- Insert rows into the "Boats" table
delete from Boats;
INSERT INTO Boats (bid, name, color) VALUES
    (101, 'Interlake', 'blue'),
    (102, 'Interlake-2', 'red'),
    (103, 'Clipper', 'green'),
    (104, 'Marine', 'red');

-- Insert rows into the "Reserves" table
delete from Reserves;
INSERT INTO Reserves (sid, bid, day) VALUES
    (22, 101, '10/10/98'),
    (22, 102, '10/11/98'),
    (22, 103, '10/8/98'),
    (22, 104, '10/7/98'),
    (31, 102, '11/10/98'),
    (31, 103, '10/6/98'),
    (31, 104, '11/12/98'),
    (64, 101, '9/5/98'),
    (64, 102, '9/8/98'),
    (74, 103, '9/8/98');



 * sqlite:///lab4_SQL.db
Done.
Done.
Done.
Done.
Done.
Done.
0 rows affected.
10 rows affected.
0 rows affected.
4 rows affected.
0 rows affected.
10 rows affected.


[]

##Class Examples


### Example 1
Use "=" in Where clause (Be careful!)

In [3]:
%%sql
Select S.sname
From Sailors S
WHERE S.sid = (
  select R.sid
  from Reserves R
  where R.bid = 103
)

 * sqlite:///lab4_SQL.db
Done.


sname
Dustin


### Example 2
Use Exists in Where clause

In [4]:
%%sql
Select S.sname
From Sailors S
WHERE exists (
  select *
  from Reserves R
  where R.bid = 103 and S.sid=R.sid
)

 * sqlite:///lab4_SQL.db
Done.


sname
Dustin
Lubber
Horatio


### Example 3
Use IN in Where clause

In [5]:
%%sql
Select S.sname
From Sailors S
WHERE S.sid in (
  select R.sid
  from Reserves R
  where R.bid = 103
)

 * sqlite:///lab4_SQL.db
Done.


sname
Dustin
Lubber
Horatio


### Example 4

ANY and ALL  (not supported in SQLite, but in Mysql)

In [6]:
%%sql

-- SQLite does not support ANY
Select S.sname
From Sailors S
WHERE S.rating > ANY (
  select S1.rating
  from Sailors S1, Reserves R1
  where S1.sid =R1.sid and R1.bid = 103
)

 * sqlite:///lab4_SQL.db
(sqlite3.OperationalError) near "select": syntax error
[SQL: -- SQLite does not support ANY
Select S.sname
From Sailors S
WHERE S.rating > ANY (
  select S1.rating 
  from Sailors S1, Reserves R1
  where S1.sid =R1.sid and R1.bid = 103 
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [7]:
%%sql

-- SQLite does not support ALL

Select S.sname
From Sailors S
WHERE S.age >= ALL (
  select S1.age
  from Sailors S1
)

 * sqlite:///lab4_SQL.db
(sqlite3.OperationalError) near "ALL": syntax error
[SQL: -- SQLite does not support ALL

Select S.sname
From Sailors S
WHERE S.age >= ALL (
  select S1.age 
  from Sailors S1 
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Example 5
subquery in From clause

In [8]:
%%sql
select SQ.sname, SQ.bname
from
  (
  select S.sname, B.name as bname
  from Sailors S, Boats B, Reserves R
  where S.sid=R.sid and B.bid=R.bid
  ) SQ
where SQ.bname='Interlake';


 * sqlite:///lab4_SQL.db
Done.


sname,bname
Dustin,Interlake
Horatio,Interlake


### Example 6
Rewriting INTERSECT Queries using IN

In [23]:
%%sql
-- set operation (no duplicates!)
select S.sid
from Sailors S, Boats B, Reserves R
where S.sid=R.sid and B.bid=R.bid and B.color='red'

INTERSECT

select S2.sid
from Sailors S2, Boats B2, Reserves R2
where S2.sid=R2.sid and B2.bid=R2.bid and B2.color='green'


 * sqlite:///lab4_SQL.db
Done.


sid
22
31


In [15]:
%%sql
select S.sid
from Sailors S, Boats B, Reserves R
where S.sid=R.sid and B.bid=R.bid and B.color='red'
and S.sid in (
  select S2.sid
  from Sailors S2, Boats B2, Reserves R2
  where S2.sid=R2.sid and B2.bid=R2.bid and B2.color='green'
)

 * sqlite:///lab4_SQL.db
Done.


sid
22
22
31
31


### Example Exercise 1
Find sids of sailors who've reserved only green boats

In [19]:
%%sql
select R.sid
from Reserves R, Boats B
where B.bid=R.bid and B.color='green'
and R.sid not in (
  select R1.sid
  from Reserves R1, Boats B1
  where B1.bid=R1.bid and B1.color<>'green'
)

 * sqlite:///lab4_SQL.db
Done.


sid
74


In [20]:
%%sql
select R.sid
from Reserves R
where not exists(
  select *
  from Reserves R1, Boats B1
  where R.sid=R1.sid
  and B1.bid=R1.bid
  and B1.color<>'green'

)



 * sqlite:///lab4_SQL.db
Done.


sid
74


### Example Exercise 2
Find sids of sailors who've the highest rating

In [22]:
%%sql
select S1.sname
from Sailors S1
where S1.sid not in (
  select S2.sid
  from Sailors S2, Sailors S3
  where S2.rating < S3.rating
)

 * sqlite:///lab4_SQL.db
Done.


sname
Rusty
Zorba


## Class Exercises


### Exercise 1
Find sailor names who’ve reserved all boats

In [9]:
%%sql



 * sqlite:///lab4_SQL.db


### Exercise 2
Find sailor names who've reserved all red boats


In [10]:
%%sql



 * sqlite:///lab4_SQL.db


### Exercise 3

Write SQL declarations for creating the schemas of the following three tables. Include necessary key constraints.

1. Employee(<u>eid</u>:integer, ename:string, age:integer, salary:real)

2. Works(<u>eid</u>:integer, <u>did</u>:integer, pct_time:integer)

3. Department(<u>did</u>:integer, dname:string, budge:real, managerid:integer)

The keys are underlined in each relation. Relation **Employee** stores employee information such as unique identifier **eid**, employee name **ename**, **age** and **salary**. Relation **Department** stores the department unique identifier **did**, department name **dname**, the department **budget** and **managerid** which is the **eid** of the employee who is managing the department. The **managerid** value must always be found in the **eid** field of a record of the Employee relation. The Works relation tracks which
employee works in which department, and what **percentage** of the time s/he allocates to that department. Note that, an employee can work in several departments.

In [11]:
%%sql

drop table if exists Employee;
drop table if exists Department;
drop table if exists Works;

-- your code goes below



 * sqlite:///lab4_SQL.db
Done.
Done.
Done.
Done.


[]

### Insert values to tables
Execute the cell below to insert values to the three tables.
1. Employee

| eid     | ename         | age | salary |  
|---------|---------------|-----|--------|
| 101     | Andrew Fuller | 40  | 50000  |
| 102     | Justin Peacock| 25  | 20000  |
| 103     | Nancy Buchanan| 35  | 30000  |
| 104     | Robert King   | 30  | 20000  |

2. Works:

| eid     | did  | pct_time |
|---------|------|----------|
| 101     | 1001 | 100      |
| 102     | 1001 | 20       |
| 102     | 1002 | 50       |
| 102     | 1003 | 10       |
| 103     | 1003 | 90       |
| 103     | 1001 | 10       |
| 104     | 1001 | 20       |
| 104     | 1002 | 70       |

3. Department:

| did     | dname    | budget  | managerid |
|---------|----------|---------|-----------|
| 1001    | IT       | 500000  | 101       |
| 1002    | Computing| 750000  | 104       |
| 1003    | Finance  | 350000  | 103       |




In [12]:
%%sql
DELETE FROM Employee;
DELETE FROM Works;
DELETE FROM Department;
insert into Employee values ( 101,'Andrew Fuller', 40, 50000);
insert into Employee values ( 102, 'Justin Peacock', 25, 20000);
insert into Employee values ( 103, 'Nancy Buchanan', 35, 30000);
insert into Employee values ( 104, 'Robert King', 30, 20000);

insert into Department values (1001, 'IT', 500000, 101);
insert into Department values (1002, 'Computing', 750000, 104);
insert into Department values (1003, 'Finance', 350000, 103);

insert into Works values (101, 1001, 100);
insert into Works values (102, 1001, 20);
insert into Works values (102, 1002, 50);
insert into Works values (102, 1003, 10);
insert into Works values (103, 1003, 90);
insert into Works values (103, 1001, 10);
insert into Works values (104, 1001, 20);
insert into Works values (104, 1002, 70);

 * sqlite:///lab4_SQL.db
(sqlite3.OperationalError) no such table: Employee
[SQL: DELETE FROM Employee;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 4
Find the name of departments who employ only persons of age 30 or younger. List each name only once.

In [13]:
%%sql




 * sqlite:///lab4_SQL.db


### Exercise 5
Find the names of employees who work in all departments

In [14]:
%%sql





 * sqlite:///lab4_SQL.db
